In [1]:
import os

In [2]:
%pwd

'c:\\Users\\Vincent\\Documents\\GitHub\\Text-Classification\\notebook'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\Vincent\\Documents\\GitHub\\Text-Classification'

In [5]:
## 3. Update the entity

from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    input_data_path: Path
    processed_data_path: Path
    X_train_data_path: Path
    X_test_data_path: Path
    y_train_data_path: Path 
    y_test_data_path: Path
    X_val_data_path: Path
    y_val_data_path: Path 

In [6]:
## 3. Update the entity

from src.constants import *
from src.utils.common import read_yaml, create_directories

In [7]:
## 4. Update the configuration manager in src config

class ConfigurationManager:
    def __init__(
        self, 
        config_filepath = CONFIG_FILE_PATH):
        
        self.config = read_yaml(config_filepath)

        create_directories([self.config.output_root])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            input_data_path=config.input_data_path,
            processed_data_path=config.processed_data_path,
            X_train_data_path=config.X_train_data_path,
            X_test_data_path=config.X_test_data_path,
            y_train_data_path=config.y_train_data_path,
            y_test_data_path=config.y_test_data_path,
            X_val_data_path=config.X_val_data_path,
            y_val_data_path=config.y_val_data_path
        )

        return data_ingestion_config

In [8]:
## 5. Update the components

import os
import sys
from src.exception import CustomException
from src import logger
import pandas as pd
from dataclasses import dataclass
from sklearn.model_selection import train_test_split
from src.utils.common import encoded
import numpy as np

In [9]:
## 5. Update the components

class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    def initiate_data_ingestion(self):
        '''
        This function is responsible for data ingestion
        
        '''
        logger.info("Data Ingestion") 
        
        try:
            logger.info("Reading the data")
            df=pd.read_csv(self.config.input_data_path)
            
            df = df[['TITLE','CATEGORY']]
            
            logger.info("Sampling 5000 rows for each category at random") 
            e = df[df['CATEGORY'] == 'e'].sample(n=5000)
            b = df[df['CATEGORY'] == 'b'].sample(n=5000)
            t = df[df['CATEGORY'] == 't'].sample(n=5000)
            m = df[df['CATEGORY'] == 'm'].sample(n=5000)

            df_selected = pd.concat([e,b,t,m], ignore_index=True)
            df_selected = df_selected.reindex(np.random.permutation(df_selected.index)).reset_index(drop=True)
            df_selected['TARGET'] = df_selected.apply(lambda x: encoded(x['CATEGORY']), axis=1)
            os.makedirs(os.path.dirname(self.config.processed_data_path),exist_ok=True)
            df_selected.to_csv(self.config.processed_data_path,index=False,header=True)   
            
            logger.info("Splitting the data into X and y")
            seed = 18
            X = df_selected['TITLE']
            y = df_selected['TARGET']
            
            logger.info("Train test split initiated to create Training, Validation and Test data")
            X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=seed)
            X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.10, random_state=seed)
            
            logger.info("Saving the data")
            os.makedirs(os.path.dirname(self.config.X_train_data_path),exist_ok=True)
            X_train.to_csv(self.config.X_train_data_path,index=False,header=True)
            X_test.to_csv(self.config.X_test_data_path,index=False,header=True)
            y_train.to_csv(self.config.y_train_data_path,index=False,header=True)
            y_test.to_csv(self.config.y_test_data_path,index=False,header=True)
            X_val.to_csv(self.config.X_val_data_path,index=False,header=True)
            y_val.to_csv(self.config.y_val_data_path,index=False,header=True)

            logger.info("Ingestion of the data is completed")
            
            return(
                    self.config.X_train_data_path,
                    self.config.X_test_data_path,
                    self.config.y_train_data_path,
                    self.config.y_test_data_path,
                    self.config.X_val_data_path,
                    self.config.y_val_data_path
                )
        
        except Exception as e:
            raise CustomException(e,sys)    

In [10]:
## 6. Update the pipeline

try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.initiate_data_ingestion()
except Exception as e:
  raise e

[2024-07-15 19:37:03,026: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-07-15 19:37:03,027: INFO: common: created directory at: output]
[2024-07-15 19:37:03,028: INFO: common: created directory at: output/data_ingestion]
[2024-07-15 19:37:03,029: INFO: 3567442147: Data Ingestion]
[2024-07-15 19:37:03,030: INFO: 3567442147: Reading the data]
[2024-07-15 19:37:04,442: INFO: 3567442147: Sampling 5000 rows for each category at random]
[2024-07-15 19:37:04,811: INFO: 3567442147: Splitting the data into X and y]
[2024-07-15 19:37:04,811: INFO: 3567442147: Train test split initiated to create Training, Validation and Test data]
[2024-07-15 19:37:04,827: INFO: 3567442147: Saving the data]
[2024-07-15 19:37:04,878: INFO: 3567442147: Ingestion of the data is completed]
